# Kijiji Scraper

### Uses BeautifulSoup and parse and extract information from the house listings in Kijiji - in this case I have chose the GTA area to scrape. Each listing is transformed into a row and then uploaded to a cloud database (MongoDB).  

Note: In my website, the code is slightly modified for simpler functionality.

In [7]:
import requests
from bs4 import BeautifulSoup
import re

Scraping inside Kijiji. First starting from the first page. Then go to each listing one by one, and then going to next page once all the listings are exhausted.

In [8]:
page = requests.get("https://www.kijiji.ca/b-house-for-sale/canada/page-3/c35l0")
soup = BeautifulSoup(page.content, 'html.parser')


#### Finding all the links for the listings

In [9]:
links = soup.find_all("a")
listings = []

for link in links:
    if 'class="title"' in str(link):
#         print(str(link))
        hit = re.findall('href="(.*?)"', str(link))
        listings.append(hit)
        
listings


        

[['/v-house-for-sale/london/20-harlequin-court-st-thomas/1636590972'],
 ['/v-house-for-sale/chatham-kent/semi-detached-raised-ranch-4bed-2bath-and-fully-fenced-54900/1640124688'],
 ['/v-house-for-sale/vancouver/ultra-luxury-townhouse-for-7-5m-est-market-value-9-5m/1640234918'],
 ['/v-house-for-sale/calgary/truman-homes-dawsons-landing-pre-con-home-sold-out-project/1639791082'],
 ['/v-house-for-sale/hamilton/rent-to-own-program-or-buy-house-with-our-zero-down-program/1598545462'],
 ['/v-maison-a-vendre/longueuil-rive-sud/reprise-de-finance-rive-sud-de-montreal-liste-gratuite/1640295427'],
 ['/v-house-for-sale/oshawa-durham-region/3br-2wr-move-in-ready-home/1640295350'],
 ['/v-house-for-sale/ottawa/home-being-sold-as-is-ottawa-area/1640295286'],
 ['/v-house-for-sale/city-of-toronto/solid-detached-home-4-3-bed-2-bath-1-5-storey-brick-home/1640295249'],
 ['/v-house-for-sale/mississauga-peel-region/14-whalecove-court-for-sale/1640170330'],
 ['/v-maison-a-vendre/longueuil-rive-sud/reprise-de

In [10]:
listings[0]

['/v-house-for-sale/london/20-harlequin-court-st-thomas/1636590972']

In [11]:
requests.get("http://kijiji.ca" + listings[0][0])

<Response [200]>

#### Testing for first listing and checking out the description, locaton, price, etc.

In [12]:
"http://kijiji.ca" + listings[0][0]

'http://kijiji.ca/v-house-for-sale/london/20-harlequin-court-st-thomas/1636590972'

In [13]:
newPage = requests.get("http://kijiji.ca" + listings[0][0])
soupTest = BeautifulSoup(newPage.content, 'html.parser')

#### Finding rooms, washrooms, size in sqft

Bedrooms, bathrooms and size is all part of one \<ul>, so I just went for the \<dd> tags inside the list, and then used regex to extract all the numbers that I want. A simpler way would be to use .string method of BeautifulSoup, but I did not know of this until after I finished making this attributeFinder() method. But I have seen other listings use different tags but did not run into that issue very often when the location is set to GTA.

In [14]:
soupp = BeautifulSoup(newPage.content, 'html.parser')

# FIX NEEDED. washrooms returning null Update: Fixed


# Returns all attributes found as a dictionary. Returns 'NA' if not found.
def attributeFinder(soup):
    attrs = []
    attrDict = {}
    attrDict['bedRooms'] = 'NA'
    attrDict['bathRooms'] = 'NA'
    attrDict['size'] = 'NA'

    # target_details = soup.find(id="mainPageContent")

    # size_rooms = soup.find(id="vip-body")
    # size_rooms = soup.find(id="AttributeList")

    size_rooms = soup.find_all("dd")
    size_rooms = list(size_rooms)

    for attribute in size_rooms:
#         print(str(attribute))
        attrs.append(re.findall('">(.*?)</', str(attribute)))
        
    if len(attrs) > 0:
        
        attrs = [item for sublist in attrs for item in sublist]
        
        i = 0
        for feature in attrs:
            
            
            if feature == '':
                continue

            if i==0:
                attrDict['bedRooms'] = attrs[0]
                i += 1

            elif i==1:
                attrDict['bathRooms'] = attrs[1]
                i += 1
            
            elif i==2:
                attrDict['size'] = attrs[2]
                i += 1
        
#         print(attrDict)
        return attrDict
        
    else:
        return 'NO'

print(attributeFinder(soupp))

{'bedRooms': '4', 'bathRooms': '3', 'size': 'NA'}


Problem: soup can't find many tags I want to, even if I reference by ID. Seems like it can only capture the static elements of the webpage. Selenium is one solution. But in this case, I have used other tags and move to the children tags step by step to work around it.

Update: Silly mistake of not updating the page variable to the new request. Fixed.

#### Finding price and address. Same as attributeFinder() but simpler because it is easy to find the price using regex or .string. It is fun to learn and use regex so I chose to stick with it for now. Price is contained inside a \<span> tag.

For some reason, the price is not displayed in the same structure if I set the location to Quebec. But, no such problem for GTA, unless the poster did not specify the price.

In [15]:
# FIX PRICES RETURNING NULL

def priceFinder(soup):

    priceSearch = soup.find_all("span")
    price = []

    for item in priceSearch:
        price = re.findall('\$(.*?)</', str(priceSearch))

    if len(price)>0:
        price = price[0]
        print(f"Fetched price: {price}")
        return(price)
    
    else:
        print("No Price Listed")
        return 0
    

In [16]:
# priceSearch = soup.find_all("span")
# price = []

# for item in priceSearch:
#     price = re.findall('\$(.*?)</', str(priceSearch))

# price = price[0]
# print(price)

In [17]:
testList = [item for sublist in listings for item in sublist]

#### Finding Adress. Following a similar pattern to the methods above...

In [18]:
currentPage = requests.get("http://kijiji.ca" + testList[1])

soup = BeautifulSoup(currentPage.content, 'html.parser')

print("http://kijiji.ca" + testList[1])

http://kijiji.ca/v-house-for-sale/chatham-kent/semi-detached-raised-ranch-4bed-2bath-and-fully-fenced-54900/1640124688


In [19]:
def addressFinder(soup):

    addressSearch = str(soup.find(id="ViewItemPage"))
    address = re.findall('itemprop="address"(.*?)</sp', str(addressSearch))
    if len(address) > 0:
        address = address[0].strip()
        address = address.split(">")[1]
        
#         print(address)
        return address
    
    else: return 'NA'
    
print(addressFinder(soup))

Fort Street, Tilbury, ON


In [20]:
addressSearch = str(soup.find(id="ViewItemPage"))
address = re.findall('itemprop="address"(.*?)</sp', str(addressSearch))
address = address[0].strip()
address = address.split(">")[1]
address

'Fort Street, Tilbury, ON'

#### Finding description. Easiest one so far. 

In [31]:
currentPage = requests.get("http://kijiji.ca" + listings[10])
print("http://kijiji.ca" + listings[10])
soup = BeautifulSoup(currentPage.content, 'html.parser')

http://kijiji.ca/v-maison-a-vendre/longueuil-rive-sud/reprise-de-finance-a-vendre-candiac-la-prairie-st-philippe/1640295140


In [32]:
def descFinder(soup):
    desc = soup.find("p").string
#     print(desc)
    return desc

print(descFinder(soup))

Reprise de Finance à Vendre Candiac - La Prairie - Saint-Philippe


In [33]:
Currentsoup = BeautifulSoup(currentPage.content, 'html.parser')

priceSearch = Currentsoup.find_all("span")
# print(priceSearch)
price = []

for item in priceSearch:
    price = re.findall('\$(.*?)</', str(priceSearch))
    
if len(price)>0:
    price = price[0]
    print(price)
else:
    print(0)

0


#### Crawling through the listings

In [22]:
import time



testList = [item for sublist in listings for item in sublist]
testList = [str(item) for item in testList]
testList = testList[0:2]


for listing in testList:
    print("Scraping through : ")
    print("http://kijiji.ca" + listing)
    
    currentPage = requests.get("http://kijiji.ca" + listing)
    
    print("request success")
    
    Currentsoup = BeautifulSoup(currentPage.content, 'html.parser')
    
    attrDict = attributeFinder(Currentsoup)
        
    price = priceFinder(Currentsoup)
    
    time.sleep(4)

Scraping through : 
http://kijiji.ca/v-house-for-sale/london/20-harlequin-court-st-thomas/1636590972
request success
Fetched price: 674,900
Scraping through : 
http://kijiji.ca/v-house-for-sale/chatham-kent/semi-detached-raised-ranch-4bed-2bath-and-fully-fenced-54900/1640124688
request success
Fetched price: 549,000


This is working for the most part. Testing it on the whole page...

In [23]:
page = requests.get("https://www.kijiji.ca/b-house-for-sale/canada/page-3/c35l0")
soup = BeautifulSoup(page.content, 'html.parser')


In [24]:
links = soup.find_all("a")
listings = []

for link in links:
    if 'class="title"' in str(link):
#         print(str(link))
        hit = re.findall('href="(.*?)"', str(link))
        listings.append(hit)
        
listings = [item for sublist in listings for item in sublist]
# listings

# Scraping through one entire page

In [34]:
page = requests.get("https://www.kijiji.ca/b-house-for-sale/canada/page-3/c35l0")
soup = BeautifulSoup(page.content, 'html.parser')


links = soup.find_all("a")
listings = []

for link in links:
    if 'class="title"' in str(link):
#         print(str(link))
        hit = re.findall('href="(.*?)"', str(link))
        listings.append(hit)
        
listings = [item for sublist in listings for item in sublist]
# listings

In [35]:
for listing in listings[0:3]:
    print("Scraping through : ")
    print("http://kijiji.ca" + listing)
    
    currentPage = requests.get("http://kijiji.ca" + listing)
    
    print("request success")
    
    Currentsoup = BeautifulSoup(currentPage.content, 'html.parser')
    
    attrDict = attributeFinder(Currentsoup)
        
    price = priceFinder(Currentsoup)
    
    address = addressFinder(Currentsoup)
    
    desc = descFinder(Currentsoup)
    
    time.sleep(6)

Scraping through : 
http://kijiji.ca/v-house-for-sale/lethbridge/judicial-sale-624-12c-street-north-lethbridge-alberta-t1h/1640067504
request success
Fetched price: 200,000
Scraping through : 
http://kijiji.ca/v-house-for-sale/burnaby-new-westminster/prime-burnaby-location-7961-burnfield-cres-move-in-today/1636663678
request success
Fetched price: 1,848,000
Scraping through : 
http://kijiji.ca/v-house-for-sale/mississauga-peel-region/corner-unit-freehold-townhouse-2-220-sqft-brampton-new-2020/1639616254
request success
Fetched price: 875,900


### Fixing atrributes not showing up

In [ ]:
# testReq = requests.get("http://kijiji.ca/v-house-for-sale/winnipeg/handyman-flipper-investor-house-for-sale-private-sale/1639156372")

In [ ]:
# newPage = requests.get("http://kijiji.ca" + listings[0][0])
# soup = BeautifulSoup(newPage.content, 'html.parser')

# attrs = []
# attrDict = {}
# attrDict['bedRooms'] = 'NA'
# attrDict['bathRooms'] = 'NA'
# attrDict['size'] = 'NA'

#     # target_details = soup.find(id="mainPageContent")

#     # size_rooms = soup.find(id="vip-body")
#     # size_rooms = soup.find(id="AttributeList")

# size_rooms = soup.find(id="AttributeList")
# print(size_rooms)
# size_rooms = list(size_rooms)

# for attribute in size_rooms:
#     attrDict = {}
#     attrDict['bedRooms'] = 'NA'
#     attrDict['bathRooms'] = 'NA'
#     attrDict['size'] = 'NA'
#     print(str(attribute))
#     attrs.append(re.findall('">(.*?)</', str(attribute)))
        
# if len(attrs) > 0:  
#     attrs = [item for sublist in attrs for item in sublist]
#     print(attrs)
    
#     i = 0
#     for feature in attrs:

            
#         if feature == '':
#             continue

#         if i==0:
#             attrDict['bedRooms'] = attrs[0]
#             i += 1

#         elif i==1:
#             attrDict['bathRooms'] = attrs[1]
#             i += 1
            
#         elif i==2:
#             attrDict['size'] = attrs[2]
#             i += 1
        
# #     print(attrDict)

        
# else:
#     print(None)

## Crawling through all listings in one page and then looping through next pages

In [27]:
page = requests.get("https://www.kijiji.ca/b-house-for-sale/gta-greater-toronto-area/c35l1700272")
soup = BeautifulSoup(page.content, 'html.parser')
nex = soup.find('a', {'title': "Next"}).get("href")

print(nex)
print(nex.startswith('/'))

/b-house-for-sale/gta-greater-toronto-area/page-2/c35l1700272?ad=offering
True


#### This generator function incorporates all created methods to fetch each piece of data from a listing. After each page is exhausted, then we move on to the next page by finding the href for the "Next" button.

It returns a dictionary containing each feature scraped for every listing

In [28]:
def crawler(startingLink):
    
    page = requests.get(startingLink)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    nextPage = ''
    
    while True:
        
        if nextPage != '':
            nextPage = soup.find('a', {'title': "Next"}).get("href")[:-11]
            print(f"Currently trying to navigate to: {nextPage}\n")
            nextPage = requests.get("https://www.kijiji.ca" + nextPage)
            soup = BeautifulSoup(nextPage.content, 'html.parser')
            print("NEXT PAGE SUCCESS\n")
    
        links = soup.find_all("a")
        listings = []

        for link in links:
            if 'class="title"' in str(link):
        #         print(str(link))
                hit = re.findall('href="(.*?)"', str(link))
                listings.append(hit)

        listings = [item for sublist in listings for item in sublist]

        
        
        
        for listing in listings:
            
            listingDetails = {}
            
            print("Scraping through : ")
            
            url = "http://kijiji.ca" + listing
            print(url)

            currentPage = requests.get("http://kijiji.ca" + listing)

            print("request success")

            Currentsoup = BeautifulSoup(currentPage.content, 'html.parser')
            
            listingDetails['url'] = url

            listingDetails['attrDict'] = attributeFinder(Currentsoup)

            listingDetails['price'] = priceFinder(Currentsoup)

            listingDetails['address'] = addressFinder(Currentsoup)

            listingDetails['desc'] = descFinder(Currentsoup)

            nextPage = True
            
            yield listingDetails
            
            print("\nLOADING NEXT LISTING...\n")
            
            time.sleep(6)
            
        

In [29]:
# for i in (crawler("https://www.kijiji.ca/b-house-for-sale/gta-greater-toronto-area/c35l1700272")):
#           print(i)

## Connecting to MongoDB and inserting scraped data

These two cells simply use the crawler() method to loop through every listing and scrape the data, and then insert each row to mongoDB. Kept the request per minute low to prevent any exceptions thrown due to the website blocking my IP for too many requests in a short timesplan.  

In [30]:
from pymongo import MongoClient

client = MongoClient("mongodb+srv://<user>:<password>@projects.1qkvd5p.mongodb.net/test")

db = client.KijijiCrawler

houses = db['House Listings']



In [ ]:
import pymongo

for listing in (crawler("https://www.kijiji.ca/b-house-for-sale/gta-greater-toronto-area/c35l1700272?sort=dateAsc")):
    
    try:
        
        row = {}
        row["_id"] = listing['url']
        row["price"] = listing['price']
        row["desc"] = listing['desc']
        row["address"] = listing['address']
        row["bedRooms"] = listing['attrDict']['bedRooms']
        row["bathRooms"] = listing['attrDict']['bathRooms']
        row["size"] = listing['attrDict']['size']

        insert = houses.insert_one(row)
        print("Insert Success!", row)
        
    except pymongo.errors.DuplicateKeyError:
        print("\nDuplicate Insertion, moving to next listing")
        continue
    except Exception:
        print("\n\nRequest attempts likely exceeded, waiting for a minute...\n")
        time.sleep(60)
        continue
    

Scraping through : 
http://kijiji.ca/v-house-for-sale/mississauga-peel-region/buy-house-with-our-exclusive-0-down-or-rent-to-own-programs/1572238043
request success
No Price Listed

Duplicate Insertion, moving to next listing

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/markham-york-region/visit-virtually-beautiful-detached-house-at-woodbridge-vaughan/1635030041
request success
Fetched price: 1,528,000

Duplicate Insertion, moving to next listing

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/markham-york-region/welcome-home-to-your-4-season-playground-in-jacksons-point/1636956861
request success
Fetched price: 1,298,000

Duplicate Insertion, moving to next listing

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/city-of-toronto/detached-bungalow-in-toronto-markland-wood-etobicoke/1640071724
request success
No Price Listed

Duplicate Insertion, moving to next listing

LOADING NEXT LISTING

Scraping through : 
http://kijiji.ca/v-house-for-sale/mississauga-peel-region/pre-constrution-town-house-suits-for-sale-in-mississauga-on/1623561953
request success
Fetched price: 1,459,900
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/mississauga-peel-region/pre-constrution-town-house-suits-for-sale-in-mississauga-on/1623561953', 'price': '1,459,900', 'desc': 'PRE-CONSTRUCTION TOWNHOME SUITS FOR SALE', 'address': 'Mississauga, ON L5H 2H3', 'bedRooms': '2 + Den', 'bathRooms': '3', 'size': '1,490'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/mississauga-peel-region/pre-constrution-town-house-suits-for-sale-in-mississauga-on/1623572585
request success
Fetched price: 1,479,900
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/mississauga-peel-region/pre-constrution-town-house-suits-for-sale-in-mississauga-on/1623572585', 'price': '1,479,900', 'desc': 'PRE-CONSTRUCTION TOWNHOME SUITS FOR SALE', 'address': 'Mississauga, ON L5H 2H3', 

Scraping through : 
http://kijiji.ca/v-house-for-sale/oshawa-durham-region/detached-house-for-sale-749-000/1632905307
request success
Fetched price: 749,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/oshawa-durham-region/detached-house-for-sale-749-000/1632905307', 'price': '749,000', 'desc': "Welcome To The Prestigious Community Of Windfields Farms. This Detached, Freshly Painted, 3 Bdrm, 3 Bthrm, One Owner Home Features 3 Car Park, No Sidewalk, Interlocked Front Yard, Exterior Pot Lights & Manicured Gardens. Enter Into The Bright & Spacious Main Floor W/ Open Concept Layout, Hardwood Floors, Pot Lights & California Shutters Throughout. Fireplace & Gorgeous Bay Window Invite You Into The Cozy Family Rm. The Modern Kitchen Has S/S Appliances, Under Cabinet Lighting, Backsplash & French Door That Lead You To Your Private Oasis. The Hardwood Staircase Leads To A Tranquil Primary Bdrm W/ 4 Pc Ensuite & Huge W/I Closet. This Turn Key Home Won't Last Long.\nWalking Distance 

request success
Fetched price: 949,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/oshawa-durham-region/whitby-home-w-3-bedrooms/1632917647', 'price': '949,000', 'desc': 'Price: $949,000/ $2727/mo*', 'address': 'Whitby, ON L1N 5T8', 'bedRooms': '3', 'bathRooms': '3', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/city-of-toronto/beautiful-home-near-leslie-lawrence/1616098097
request success
Fetched price: 2,222,888
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/city-of-toronto/beautiful-home-near-leslie-lawrence/1616098097', 'price': '2,222,888', 'desc': "Here's an opportunity to buy a sparkling property in one of the most desirable areas of Toronto!\n*** Price Has Been reduced. ***\nLocated near Lawrence and Leslie.\nI am a BUYER'S Agent and will get you the best possible price! Please call me to discuss details. Closing is available for July or August\nLot size is about 50 feet X 110 feet.", 'address': 'Toronto

Scraping through : 
http://kijiji.ca/v-house-for-sale/mississauga-peel-region/3br-3wr-semi-detach-in-mississauga-near-burnhamthorpe-folkw/1632924985
request success
Fetched price: 999,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/mississauga-peel-region/3br-3wr-semi-detach-in-mississauga-near-burnhamthorpe-folkw/1632924985', 'price': '999,000', 'desc': '"3 BR 3 WR Semi-Detach... in Mississauga, near Burnhamthorpe & Folkway area.', 'address': 'Mississauga, ON L5L 2C2', 'bedRooms': '3', 'bathRooms': '3', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/mississauga-peel-region/4br-3wr-semi-detach-in-brampton-near-bovaird-kennedy/1632925131
request success
Fetched price: 999,800
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/mississauga-peel-region/4br-3wr-semi-detach-in-brampton-near-bovaird-kennedy/1632925131', 'price': '999,800', 'desc': '"4 BR 3 WR Semi-Detach... in Brampton, near Bovaird / Kennedy area.', 'addre

...

Scraping through : 
http://kijiji.ca/v-house-for-sale/city-of-toronto/townhouse-for-sale/1633161522
request success
Fetched price: 899,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/city-of-toronto/townhouse-for-sale/1633161522', 'price': '899,000', 'desc': 'Price: $899,000/ $2583/mo*', 'address': 'Toronto, ON M6P 4A1', 'bedRooms': '2', 'bathRooms': '3', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/oshawa-durham-region/fully-detached-legal-duplex-in-central-oshawa/1633163573
request success
Fetched price: 749,900
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/oshawa-durham-region/fully-detached-legal-duplex-in-central-oshawa/1633163573', 'price': '749,900', 'desc': 'Price: $749,900/ $2155/mo*', 'address': 'Oshawa, ON L1H 5Z4', 'bedRooms': '3', 'bathRooms': '3', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/markham-york-region/maple-vaughan-3-bed-3-bath/1633164725

request success
No Price Listed
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/city-of-toronto/buy-home-with-zero-5-down-or-rent-to-own-program/1633171618', 'price': 0, 'desc': "Welcome To 4 + 2 Bedrooms Detached Home In High Demand Area. New Roof (2021), New Kitchen (2022),New Counters In All Washrooms, New Pot-Lights In Entire Home, Newly Painted Home. Double Door Entry, 9 Ft Ceilings On Main Floor, 3 Full Baths On 2nd Floor. Two Bedroom Finished Basement And Separate Entrance From The Builder. No Carpet In The Home. Close To Fortino, All Major Banks, Hospital, Public Transit, Parks & Schools, Medical Offices, Retail Plazas.\nExtras: S/S (2 Fridge, Stove, Dishwasher ) 1 White Stove In Bsmt., Washer, Dryer, All Window Covering, All Elf's , Garage Door Opener.", 'address': 'Chambers Ave, Toronto, York, Ontario M6N 3M5', 'bedRooms': '4', 'bathRooms': '3', 'size': '2,300'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/city-of-toronto/2-1-bedr

request success
Fetched price: 539,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/oshawa-durham-region/3-bedrooms-lakeview-oshawa/1633180679', 'price': '539,000', 'desc': 'Price: $539,000/ $1549/mo*', 'address': 'Oshawa, ON L1H 3R1', 'bedRooms': '3', 'bathRooms': '1', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/mississauga-peel-region/amazing-3-bed-2-bath-freehold-town-in-georgetown/1633180917
request success
Fetched price: 847,900
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/mississauga-peel-region/amazing-3-bed-2-bath-freehold-town-in-georgetown/1633180917', 'price': '847,900', 'desc': "Welcome Home to 5 Bradley Drive in Georgetown (Halton Hills). This meticulously maintained home has upgrades from top to bottom! Nothing to do but move In, and it's your very own. Open concept main floor with hardwood floor and barnboard accent wall. 3 spacious bedrooms upstairs, 2 baths and a professionally finished basem

Scraping through : 
http://kijiji.ca/v-house-for-sale/oshawa-durham-region/oshawa-living-3-1-bdrm-2-bth-hillside-ave-park-rd/1633194922
request success
Fetched price: 775,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/oshawa-durham-region/oshawa-living-3-1-bdrm-2-bth-hillside-ave-park-rd/1633194922', 'price': '775,000', 'desc': 'Price: $775,000/ $2227/mo*', 'address': 'Oshawa, ON L1J 1T5', 'bedRooms': '3', 'bathRooms': '2', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/oshawa-durham-region/pickering-3-1-bdrm-2-bth-bayly-liverpool/1633195114
request success
Fetched price: 829,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/oshawa-durham-region/pickering-3-1-bdrm-2-bth-bayly-liverpool/1633195114', 'price': '829,000', 'desc': 'Price: $829,000/ $2382/mo*', 'address': 'Pickering, ON L1W 1H2', 'bedRooms': '3', 'bathRooms': '2', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-fo

request success
Fetched price: 1,399,800
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/city-of-toronto/meadowvale-ellesmere-for-sale/1633201708', 'price': '1,399,800', 'desc': 'Price: $1,399,800 /$4023/mo', 'address': 'Scarborough, ON M1C 4S6', 'bedRooms': '6+', 'bathRooms': '4', 'size': 'NA'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/oakville-halton-region/tiny-home-mobile/1633202168
request success
Fetched price: 65,000
Insert Success! {'_id': 'http://kijiji.ca/v-house-for-sale/oakville-halton-region/tiny-home-mobile/1633202168', 'price': '65,000', 'desc': 'Custom Built Tiny Home\n- 28ft x 14ft\n- 13 ft ceilings\n- bathroom skylight\n- built on a brand new trailer\n(considered a non permanent structure)\n- brand new build', 'address': 'Oakville, ON L6H 0A8', 'bedRooms': '1', 'bathRooms': '1', 'size': '433'}

LOADING NEXT LISTING...

Scraping through : 
http://kijiji.ca/v-house-for-sale/city-of-toronto/mccowan-and-kingston-for-sale-in

ML Ideas. 

- Determine Price based on features.

- Determine if a posting is an advertisement or not with NLP

- Predict the size of house based on features